# 动态规划

动态规划解题分三步：
1. 找出状态转移方程
2. 初始化数组
3. 更新数组

今天总结一些经典的题目：
- [最大子序和](https://leetcode-cn.com/problems/maximum-subarray/)
- [买卖股票的最佳时机IV](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-iv/)

## 最大子序和
转移方程: $dp[i] = max(dp[i - 1], 0) + nums[i]$

dp[i]代表前一个值和 0 的最大值加上 nums[i]的值，$\max dp_{i}$就是最大子序和

先实现一个版本，时间复杂度O(n)，空间复杂度O(n)

In [3]:
class Solution:
    def maxSubArray(self, nums) -> int:
        n = len(nums)
        dp = [0 for _ in range(n)]
        
        # 初始化，数组中第一个元素的最大和就是第一个元素值
        dp[0] = nums[0]
        ans = nums[0]

        # 状态转移：dp[i] = max(dp[i - 1], 0) + nums[i]
        for i in range(1, n):
            dp[i] = max(dp[i - 1], 0) + nums[i]
            # 更新
            ans = max(ans, dp[i]) 
        return ans
if __name__ == '__main__':
    s = Solution()
    nums = [-2,1,-3,4,-1,2,1,-5,4]
    print(s.maxSubArray(nums))

6


一般来说，根据状态转移方程，大概就知道时间复杂度是多少，但是，空间复杂度可以通过技巧减小。本题不需要存储之前的状态，只需要用一个单位保存上一个状态即可，空间复杂度O(1)如下：

In [ ]:
class Solution:
    def maxSubArray(self, nums) -> int:
        n = len(nums)
        pre = nums[0]
        max_res = nums[0]
        for i in range(1, n):
            max_res = max(max_res, pre + nums[i], nums[i])
            pre = max(pre + nums[i], nums[i])
        return max_res

if __name__ == '__main__':
    s = Solution()
    nums = [-2,1,-3,4,-1,2,1,-5,4]
    print(s.maxSubArray(nums))

## 买卖股票的最佳时机
买卖股票是一系列题目，包括进行一次交易，进行两次交易，进行多次交易，存在“冻结期”等限制条件。一次交易与上题类似，我们先看一下两次交易，再看k次交易，再看一些限制条件。
- [买卖股票的最佳时机](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock/)
- [买卖股票的最佳时机II](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-ii/)
- [买卖股票的最佳时机III](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-iii/)
- [买卖股票的最佳时机IV](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-iv/)
- [买卖股票的最佳时机含冷冻期](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-with-cooldown/)

### 两次交易
**状态转移方程**

由于允许最多交易2次，假如用dp[i]表示第i天的收益，那dp[i+1]能否用dp[i]表示？不能，所以一阶dp是不够的！

仔细想一想，问题在于交易2次，想个法子把交易次数加到dp状态转移里面，用dp[i][k]表示第i天交易k次的收益，现在尝试着写状态转移方程，可以先思考有哪些情况,很简单，无非是无操作、买、卖三种，具体分析一下：
1. 无操作。即dp[i][k] = dp[i-1][k]；
2. 买。今天买股票，说明之前没有买，收益和无操作是相同的；
3. 卖。假设手里有第j天购买的股票，今天把它卖了，即
$$
dp[i][k] = dp[j][k-1] + (pricses[i] - prices[j])
$$

综上：
$$
dp[i][k] = max(dp[i-1][k], prices[i] - prices[j] + dp[j][k-1]), j \to (0, i)
$$

时间复杂度是$O(kn^2)$

**初始化条件**
每天的收益全部初始化为0即可

In [7]:
class Solution:
    def maxProfit(self, prices) -> int:
        n = len(prices)
        if n in [0, 1]:
            return 0
        dp = [[0 for _ in range(3)] for _ in range(n)]

        for k in range(1, 3):
            for i in range(1, n):
                # 一开始手滑，写成这样，是错的，并没有保存0-j的最大收益
                # for j in range(0, i):
                #    dp[i][k] = max(dp[i-1][k], dp[j][k-1] + prices[i] - prices[j])
                prof = 0
                for j in range(0, i):
                    prof = max(prof, dp[j][k - 1] + prices[i] - prices[j])
                dp[i][k] = max(dp[i - 1][k], prof)

        return max(dp[-1][1], dp[-1][2])

if __name__ == '__main__':
    s = Solution()
    print(s.maxProfit([1, 2, 3, 4, 5]))

4


这样做，还是会超出时间限制，其实，我们只需要保存$prices[i] - prices[j] + dp[j][k-1]$的最大值，即在一次循环中保存$\max (- prices[j] + dp[j][k-1])$，时间复杂度降到$O(kn)$

这里要注意初始化条件：


In [9]:
class Solution:
    def maxProfit(self, prices) -> int:
        n = len(prices)
        dp = [[0 for _ in range(3)] for _ in range(n)]

        for k in range(1, 3):
            prof = dp[0][k - 1] - prices[0] # 初始化条件
            for i in range(1, n):
                prof = max(prof, dp[i][k - 1] - prices[i])
                dp[i][k] = max(dp[i - 1][k], prices[i] + prof)
        return max(dp[-1][1], dp[-1][2])

if __name__ == '__main__':
    s = Solution()
    print(s.maxProfit([1, 2, 3, 4, 5]))

4


### 一般化 -- K次交易

买卖K次股票是最所有题目的理论核心，参考题解，用[状态转移来实现三维DP](https://leetcode-cn.com/problems/best-time-to-buy-and-sell-stock-iv/solution/yi-ge-tong-yong-fang-fa-tuan-mie-6-dao-gu-piao-w-5/)